In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship2.git

Cloning into 'DS_Internship2'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 204 (delta 38), reused 74 (delta 21), pack-reused 111 (from 1)
Receiving objects: 100% (204/204), 47.00 MiB | 12.02 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
import zipfile
import os

os.makedirs("/content/various_sentences/", exist_ok=True)
# from https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus/
# locally task2/dataset/ner.csv.zip

with zipfile.ZipFile("/content/DS_Internship2/task2/dataset/raw/ner.csv.zip", "r") as zf:
  zf.extractall("/content/various_sentences/")


In [ ]:
import pandas as pd
import ast

def load_csv_data(data_path):

    df = pd.read_csv(data_path)

    df.dropna(inplace=True)
    print("Number of rows : ",df.shape[0]," and the number of columns : ",df.shape[1])

    return df

ner_df = load_csv_data("/content/various_sentences/ner.csv")
# necessary to convert to usable format for df
def preprocess_ner_df(df):
    for i in range(len(df)):
        pos = ast.literal_eval(df.loc[i, 'POS'])
        tags = ast.literal_eval(df.loc[i, 'Tag'])
        df.loc[i, 'POS'] = [str(word) for word in pos]
        df.loc[i, 'Tag'] = [str(word.upper()) for word in tags]
    return df

ner_df = preprocess_ner_df(ner_df)
# i need only sentences, annotations
ner_df = ner_df[['Sentence','Tag']]

ner_df.head()

Number of rows :  47959  and the number of columns :  4


,Sentence,Tag
0,Thousands of demonstrators have marched throug...,"[O, O, O, O, O, O, B-GEO, O, O, O, O, O, B-GEO..."
1,Families of soldiers killed in the conflict jo...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,They marched from the Houses of Parliament to ...,"[O, O, O, O, O, O, O, O, O, O, O, B-GEO, I-GEO..."
3,"Police put the number of marchers at 10,000 wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,The protest comes on the eve of the annual con...,"[O, O, O, O, O, O, O, O, O, O, O, B-GEO, O, O,..."


In [ ]:
tag_values = list(ner_df.Tag.values)
sentences = list(ner_df.Sentence.values)
del ner_df
print(tag_values[:5])
print(sentences[:5])

[['O', 'O', 'O', 'O', 'O', 'O', 'B-GEO', 'O', 'O', 'O', 'O', 'O', 'B-GEO', 'O', 'O', 'O', 'O', 'O', 'B-GPE', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GEO', 'I-GEO', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GEO', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-GPE', 'O', 'O', 'O', 'B-GEO', 'O']]
['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .', 'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "', 'They marched from the Houses of Parliament to a rally in Hyde Park .', 'Police put the

In [ ]:
unique_tags = set([tag for subl in tag_values for tag in subl])
print(unique_tags)

{'I-GPE', 'B-GPE', 'I-PER', 'B-EVE', 'I-ORG', 'I-NAT', 'B-NAT', 'B-PER', 'B-GEO', 'B-ORG', 'I-EVE', 'I-ART', 'I-GEO', 'B-ART', 'I-TIM', 'O', 'B-TIM'}



    geo = Geographical Entity

    org = Organization

    per = Person

    gpe = Geopolitical Entity

    tim = Time indicator

    art = Artifact

    eve = Event

    nat = Natural Phenomenon

In [ ]:
del tag_values
print(len(sentences))

47959


In [ ]:
import random

random.shuffle(sentences)
sentences = random.sample(sentences, 10000)

In [ ]:
labels = ["O", "B-ANIMAL", "I-ANIMAL"]
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

In [ ]:
def preprocess_for_ner(dataset):
    ner_data = []

    for entry in dataset:
        animal_name = entry["animal"]
        sentences = entry["sentences"]

        for sentence in sentences:
            tokens = re.findall(r'\w+|[^\w\s]', sentence, re.UNICODE)
            labels = [0] * len(tokens)  # Start with all '0' (Outside)

            # Regex to find animal names and label them
            animal_regex = re.escape(animal_name)
            for match in re.finditer(animal_regex, sentence):
                start, end = match.span()

                start_word = len(re.findall(r'\w+|[^\w\s]', sentence[:start], re.UNICODE))
                end_word = len(re.findall(r'\w+|[^\w\s]', sentence[:end], re.UNICODE))

                # Update labels using the few-nerd convention
                labels[start_word] = 1  # B-ANIMAL
                for i in range(start_word + 1, end_word):
                    labels[i] = 2  # I-ANIMAL

            ner_data.append({
                "sentence": sentence,
                "tokens": tokens,
                "labels": labels
            })

    return ner_data

In [ ]:
import re
import random
import math
import json

def tokenize_and_label_regular(sentence, tokens, labels):
    sentence_tokens = re.findall(r'\w+|[^\w\s]', sentence, re.UNICODE)
    tokens.extend(sentence_tokens)
    labels.extend(str(label2id["O"]) * len(sentence_tokens))

def process_sentence(sentence):
    tokens, labels = [], []
    tokenize_and_label_regular(sentence, tokens, labels)
    return tokens, labels

def process_entry(entry):
    tokens, labels = process_sentence(entry)
    return (tokens, labels)

non_animal_data = []
for entry in sentences:
  if entry:
    non_animal_data.append(process_entry(entry))
del sentences
random.shuffle(non_animal_data)

In [ ]:
print(non_animal_data[0][0])
print(non_animal_data[0][1])
print(non_animal_data[4][0])
print(non_animal_data[4][1])

['German', 'authorities', 'say', 'the', 'suspect', 'is', 'one', 'of', 'two', 'Lebanese', 'men', 'accused', 'of', 'planting', 'suitcase', 'bombs', 'on', 'trains', 'in', 'Cologne', 'July', '31', '.']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['The', 'official', 'Xinhua', 'news', 'agency', 'said', 'Monday', 'that', 'Beijing', 'registered', 'more', 'than', '22', ',', '000', 'new', 'vehicles', 'in', 'the', 'first', '18', 'days', 'of', 'this', 'year', '.']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [ ]:
# this is just result of notebook:  task2/notebooks/nlp/animals_900_synthetic.ipynb
with zipfile.ZipFile("/content/DS_Internship2/task2/dataset/raw/generated.zip", "r") as zf:
  zf.extractall("/content/synthetic_batch1/")

# this is result of notebook: task2/notebooks/nlp/failed_animals_synthetic.ipynb
# failed animals because of increased temperature i got invalid outputs in like 20% of cases
with zipfile.ZipFile("/content/DS_Internship2/task2/dataset/raw/generated1.zip", "r") as zf:
  zf.extractall("/content/synthetic_batch2/")

# this is result of notebook that used general animal names that people use more often
# notebook: task2/notebooks/nlp/general_names_synthetic.ipynb
with zipfile.ZipFile("/content/DS_Internship2/task2/dataset/raw/generated2.zip", "r") as zf:
  zf.extractall("/content/synthetic_batch3/")

In [ ]:
import json

# animals in sentences - separated by ||:
# "The majestic ||bald eagle|| soars high above the tranquil lake, its keen eyes scanning for prey.",

# for each 10 sentences (output from Llama) we need to extract animals
# we label as animal everything that model outputs wrapped in || (
# you can see that there's no false positives by prints that will appear in cells below)

def get_v_bar_start_indexes(string):
    """Find starting indexes of double vertical bars (||) in the string."""
    return [match.start() for match in re.finditer(r'\|\|', string)]

def check_acceptable_distortion(entity_with_markers, entity_name, is_conf):
    acceptable_distortions = [entity_name + "'s", entity_name + ",", entity_name + "'s,", entity_name + ".", entity_name]
    if is_conf:
        acceptable_distortions += [entity_name.split()[1]] if len(entity_name.split()) > 1 else []
    return entity_with_markers in acceptable_distortions

def tokenize_and_label_regular(sentence, tokens, labels):
    sentence_tokens = re.findall(r'\w+|[^\w\s]', sentence, re.UNICODE)
    tokens.extend(sentence_tokens)
    labels.extend([str(label2id["O"])] * len(sentence_tokens))

def tokenize_and_label_entity(string, tokens, labels, entity_name, is_conf):
    entity = re.sub(r"\|\|", "", string).strip()
    tokenized_entity = re.findall(r'\w+|[^\w\s]', entity, re.UNICODE)
    if check_acceptable_distortion(entity, entity_name, is_conf):
        for i, token in enumerate(tokenized_entity):
            if token in {"'s", ",", "."}:
                tokens.append(token)
                labels.append(str(label2id["O"]))
            else:
                tokens.append(token)
                labels.append(str(label2id["B-ANIMAL"]) if i == 0 else str(label2id["I-ANIMAL"]))
    else:
        print("---------------Non acceptable distortion-------------------")
        print(entity)
        tokens.extend(tokenized_entity)
        labels.extend([str(label2id["B-ANIMAL"])] + [str(label2id["I-ANIMAL"])] * (len(tokenized_entity) - 1))

def process_sentence(sentence, entity_name, is_conf):
    tokens, labels = [], []
    entity_name = entity_name.strip()
    v_bar_starting_indexes = get_v_bar_start_indexes(sentence)

    if len(v_bar_starting_indexes) % 2 != 0:
        print(f"Error: Uneven count of bars in sentence: {sentence}")
        return None

    left = 0
    for i in range(0, len(v_bar_starting_indexes), 2):
        start, end = v_bar_starting_indexes[i], v_bar_starting_indexes[i + 1] + 2
        if left < start:
            tokenize_and_label_regular(sentence[left:start], tokens, labels)
        elev = sentence[start:end]
        tokenize_and_label_entity(elev, tokens, labels, entity_name, is_conf)
        left = end
    if left < len(sentence):
        tokenize_and_label_regular(sentence[left:], tokens, labels)

    if not tokens or not labels or len(tokens) != len(labels):
        print(f"Invalid tokens or labels generated for sentence: {sentence}")
        return None

    return tokens, labels

def process_entry(entry, is_conf):
    sentences = []
    for obj in entry:
        if not obj or not isinstance(obj, dict):
            print(f"Invalid object in entry: {obj}")
            continue
        for name, sentences_list in obj.items():
            name = name.replace("|", "")
            if not isinstance(sentences_list, list):
                print(f"Invalid sentences list for name '{name}': {sentences_list}")
                continue
            for sentence in sentences_list:
                processed_sentence = process_sentence(sentence, name, is_conf)
                if processed_sentence:
                    sentences.append(processed_sentence)
                else:
                    print(f"Failed to process sentence: {sentence}")
    return sentences if sentences else None

# Process synthetic data
synthetic_data = []
synthetic_data_dir = "/content/synthetic_batch1/"
if os.path.exists(synthetic_data_dir):
    for json_file in os.listdir(synthetic_data_dir):
        filename = os.path.join(synthetic_data_dir, json_file)
        with open(filename, "r") as f:
            try:
                synthetic_data += json.load(f)
            except json.JSONDecodeError as e:
                print(f"Failed to load JSON file {json_file}: {e}")
else:
    print(f"Synthetic data directory not found: {synthetic_data_dir}")

converted_synthetic_data = []
for entry in synthetic_data:
    if entry:
        processed = process_entry(entry, False)
        if processed:
            converted_synthetic_data.extend(processed)

random.shuffle(converted_synthetic_data)
val_len = math.ceil(len(converted_synthetic_data) * 0.15)
train_len = len(converted_synthetic_data) - val_len

print(f"Train size: {train_len}, Validation size: {val_len}")


---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
bobwhite
---------------Non acceptable distortion-------------------
Spiders
---------------Non acceptable distortion-------------------
Spiders
---------------Non acceptable distortion-------------------
Spotted glass frog
---------------Non acceptable distortion-------------------
Glass frog
---------------Non accep

In [ ]:
print(converted_synthetic_data[666][0])
print(converted_synthetic_data[666][1])

['slender', 'crab', 'crab', 'spider', 'has', 'exceptional', 'eyesight', ',', 'which', 'it', 'utilizes', 'to', 'spot', 'prey', 'in', 'the', 'vast', 'environment', '.']
['1', '2', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [ ]:
print(converted_synthetic_data[111][0])
print(converted_synthetic_data[111][1])

['As', 'expert', 'diggers', ',', 'the', 'spinycheek', 'crayfish', ',', 'plays', 'a', 'vital', 'role', 'in', 'maintaining', 'the', 'balance', 'of', 'marine', 'ecosystems', '.']
['0', '0', '0', '0', '0', '1', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [ ]:
synthetic_data_batch2 = []
synthetic_data_batch2_dir = "/content/synthetic_batch2/"

# Load JSON files from batch 2 directory
if os.path.exists(synthetic_data_batch2_dir):
    for json_file in os.listdir(synthetic_data_batch2_dir):
        filename = os.path.join(synthetic_data_batch2_dir, json_file)
        try:
            with open(filename, "r") as f:
                synthetic_data_batch2 += json.load(f)
        except json.JSONDecodeError as e:
            print(f"Failed to load JSON file {json_file}: {e}")
else:
    print(f"Synthetic batch 2 directory not found: {synthetic_data_batch2_dir}")

# Process entries from batch 2
converted_synthetic_data_batch2 = []
for entry in synthetic_data_batch2:
    if entry:
        processed_entry = process_entry(entry, False)
        if processed_entry:
            # Ensure all processed entries are valid
            valid_entries = [pe for pe in processed_entry if pe and len(pe) == 2]
            if valid_entries:
                converted_synthetic_data_batch2.extend(valid_entries)
            else:
                print(f"Invalid processed entry: {processed_entry}")
        else:
            print(f"Failed to process entry: {entry}")

# Shuffle and split batch 2 data
random.shuffle(converted_synthetic_data_batch2)
val_len_batch2 = math.ceil(len(converted_synthetic_data_batch2) * 0.15)
train_len_batch2 = len(converted_synthetic_data_batch2) - val_len_batch2

print(f"Batch 2 - Train size: {train_len_batch2}, Validation size: {val_len_batch2}")


---------------Non acceptable distortion-------------------
Wild water buffalo
---------------Non acceptable distortion-------------------
Wild water buffalo
---------------Non acceptable distortion-------------------
Snapping shrimp
Invalid sentences list for name 'animals': {'fragile montaguia': ['||fragile montaguia|| was carefully selected for its unique properties', 'The characteristics of the newly identified ||fragile montaguia|| amazed scientists.', 'From ancient stories to present-day research, the fascination with ||fragile montaguia|| has been timeless.', 'A new discovery made headlines, describing a newly found ||fragile montaguia|| in a remote location.', 'Researchers use ||fragile montaguia|| in their experiments due to its delicate features.', 'The features of the ||fragile montaguia|| were documented by the expert for future generations.', 'A newly published study discussed the peculiarities of the newly observed ||fragile montaguia||.', 'The preservation of the habitat

In [ ]:
print(converted_synthetic_data_batch2[111][0])
print(converted_synthetic_data_batch2[111][1])

['This', 'hardy', 'oregon', 'shore', 'crab', 'can', 'even', 'survive', 'out', 'of', 'the', 'water', 'for', 'short', 'periods', '.']
['0', '0', '1', '2', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [ ]:
# Process synthetic data
synthetic_batch3_data = []
synthetic_batch3_dir = "/content/synthetic_batch3/"
if os.path.exists(synthetic_batch3_dir):
    for json_file in os.listdir(synthetic_batch3_dir):
        filename = os.path.join(synthetic_batch3_dir, json_file)
        with open(filename, "r") as f:
            try:
                synthetic_batch3_data += json.load(f)
            except json.JSONDecodeError as e:
                print(f"Failed to load JSON file {json_file}: {e}")
else:
    print(f"Synthetic data directory not found: {synthetic_batch3_dir}")

converted_synthetic_batch3_data = []
for entry in synthetic_batch3_data:
    if entry:
        processed = process_entry(entry, False)
        if processed:
            converted_synthetic_batch3_data.extend(processed)

random.shuffle(converted_synthetic_batch3_data)
batch3_val_len = math.ceil(len(converted_synthetic_batch3_data) * 0.15)
batch3_train_len = len(converted_synthetic_batch3_data) - batch3_val_len

print(f"Batch 3 train size: {batch3_train_len}, Validation size: {batch3_val_len}")

---------------Non acceptable distortion-------------------
oysters
---------------Non acceptable distortion-------------------
Wallaby
---------------Non acceptable distortion-------------------
beetles
---------------Non acceptable distortion-------------------
Whale
---------------Non acceptable distortion-------------------
Bison
---------------Non acceptable distortion-------------------
Waxbill
---------------Non acceptable distortion-------------------
Tarantula
---------------Non acceptable distortion-------------------
Tarantulas
---------------Non acceptable distortion-------------------
Railroads
---------------Non acceptable distortion-------------------
Railroads
---------------Non acceptable distortion-------------------
sparrows
---------------Non acceptable distortion-------------------
Sparrows
---------------Non acceptable distortion-------------------
sparrows
---------------Non acceptable distortion-------------------
sparrows
---------------Non acceptable distortio

In [ ]:
val_samples = converted_synthetic_data[:val_len]
train_samples = converted_synthetic_data[val_len:]

val_samples += converted_synthetic_data_batch2[:val_len_batch2]
train_samples += converted_synthetic_data_batch2[val_len_batch2:]

val_samples += converted_synthetic_batch3_data[:batch3_val_len]
train_samples += converted_synthetic_batch3_data[batch3_val_len:]

In [ ]:
train_len = len(train_samples)
val_len = len(val_samples)
print(train_len, val_len)

9391 1659


i didn't provide notebooks but choise of not adding non-animal sentences improved f-score and recall

In [ ]:
# non_animal_data = random.sample(non_animal_data, train_len + val_len)
# val_samples += non_animal_data[:val_len]
# train_samples += non_animal_data[:train_len]

# print(len(train_samples), len(val_samples))

In [ ]:
def save_to_json(samples, output_path):
    """
    Save the tokenized samples to a JSON file for fine-tuning.
    Each entry includes `tokens` and `labels`.
    """

    formatted_data = [{"tokens": tokens, "labels": labels} for tokens, labels in samples]
    with open(output_path, "w") as f:
        json.dump(formatted_data, f, indent=4)

output_path = "/content/training_dataset.json"
save_to_json(train_samples, output_path)

output_path = "/content/val_dataset.json"
save_to_json(val_samples, output_path)
